<a href="https://colab.research.google.com/github/mohameddhameem/IBM-Machine-Learning/blob/master/Deep%20Learning%20and%20Reinforcement%20Learning/Week%206/05i_DEMO_Reinforcement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Foundation

## Course 5, Part i: Reinforcement Learning DEMO

## Reinforcement Learning Example

In this example from Reinforcement Learning, the task is to use tools from Machine Learning to predict how an agent should act. We will then use those predictions to drive the behavior of the agent. Ideally, our intelligent agent should get a much better score than a random agent.

## Key concepts:

- **Observation**: These are the states of the game. It describes where the agent currently is.
- **Action**: These are the moves that the agent makes.
- **Episode**: One full game played from beginning (`env.reset()`) to end (when `done == True`).
- **Step**: Part of a game that includes one action. The game transitions from one observation to the next.

## Setup

This exaple uses the Python library [OpenAI Gym](https://gym.openai.com/docs/).

If you want to install everything (gym can run atari games.) follow [these instructions](https://github.com/openai/gym#installing-everything).

Now we can build an environment using OpenAI. 

In [14]:
import gym
import pandas
import numpy as np
from tqdm import tqdm

# The first part of the game uses the environment FrozenLake-V0

This is a small world with 16 tiles. 

    SFFF
    FHFH
    FFFH
    HFFG

The game starts at the S tile. The object of the game is to get to the goal (G) without landing in a hole (H).

In [2]:
# Build an environment with gym.make()
env = gym.make('FrozenLake-v0') # build a fresh environment

# Start a new game with env.reset()
current_observation = env.reset() # this starts a new "episode" and returns the initial observation

#the current observation is just the current location
print(current_observation) # observations are just a number

0


In [3]:
# we can print the environment if we want to look at it
env.render() 


SFFF
FHFH
FFFH
HFFG


In [4]:
# the action space for this environment includes four discrete actions

print(f"our action space: {env.action_space}")

new_action = env.action_space.sample() # we can randomly sample actions

print(f"our new action: {new_action}") # run this cell a few times to get an idea of the action space
# what does it look like?

our action space: Discrete(4)
our new action: 3


In [5]:
# now we act! do this with the step function

new_action = env.action_space.sample()

observation, reward, done, info = env.step(new_action)

# here's a look at what we get back
print(f"observation: {observation}, reward: {reward}, done: {done}, info: {info}")

env.render() 

observation: 1, reward: 0.0, done: False, info: {'prob': 0.3333333333333333}
  (Right)
SFFF
FHFH
FFFH
HFFG


In [6]:
# we can put this process into a for-loop and see how the game progresses

current_observation = env.reset() # start a new game

for i in range(5): # run 5 moves

    new_action = env.action_space.sample() # same a new action

    observation, reward, done, info = env.step(new_action) # step through the action and get the outputs

    # here's a look at what we get back
    print(f"observation: {observation}, reward: {reward}, done: {done}, info: {info}")

    env.render() 

observation: 0, reward: 0.0, done: False, info: {'prob': 0.3333333333333333}
  (Left)
SFFF
FHFH
FFFH
HFFG
observation: 0, reward: 0.0, done: False, info: {'prob': 0.3333333333333333}
  (Down)
SFFF
FHFH
FFFH
HFFG
observation: 0, reward: 0.0, done: False, info: {'prob': 0.3333333333333333}
  (Left)
SFFF
FHFH
FFFH
HFFG
observation: 4, reward: 0.0, done: False, info: {'prob': 0.3333333333333333}
  (Right)
SFFF
FHFH
FFFH
HFFG
observation: 0, reward: 0.0, done: False, info: {'prob': 0.3333333333333333}
  (Right)
SFFF
FHFH
FFFH
HFFG


Now we can guess what each of the outputs mean. 

**Observation** refers to the number of the tile. The tiles appear to be numbered

    0 1 2 3
    4 5 ...
    
**Reward** refers to the outcome of the game. We get 1 if we win, zero otherwise.

**Done** tells us if the game is still going. It goes to true when we win or fall into a hole.

**info** gives extra info about the world. Here, it's probabilities. Can you guess what this means here? Perhaps the world is a bit noisy.

In [7]:
# Here's how to simulate an entire episode
# We're going to stop rendering it every time to save space
# try running this a few. Does it ever win?

current_observation = env.reset()
done = False

while not done:    
    new_action = env.action_space.sample()
    new_observation, reward, done, info = env.step(new_action)
    print(f"action:{new_action} observation: {new_observation}, reward: {reward}, done: {done}, info: {info}")

action:1 observation: 1, reward: 0.0, done: False, info: {'prob': 0.3333333333333333}
action:3 observation: 1, reward: 0.0, done: False, info: {'prob': 0.3333333333333333}
action:0 observation: 5, reward: 0.0, done: True, info: {'prob': 0.3333333333333333}


Things to think about:
- What things do you notice about how the environment and actions work?
- What do you think the actions mean?
- When the agent performs the same action from the same place (same observation), does the same outcome happen every time?

The environment has some squares that always end the game (`H` in the render), some that don't (`F`), and one that is presumably the reward, if you get to it.

The actions seem like up, down, left right. But they also seem stochastic. There seems to be a 1/3 chance of going into 3 different squares with each action. 

# Part 1: Gather data

We want to build an intelligent actor but first we have to gather data on which actions are useful.

Use the above code as reference. Run a *random* agent through 1,000 or more episodes and collect data on each step.

I recommend you store this data in a pandas dataframe. Each row should be a step. Your features should include the following features or similar 

- `observation` the observation at the beginning of the step (before acting!)
- `action` the action randomly sampled
- `current_reward` the reward received after the action was performed

After you generate this data, it is recommended that you compute a column (e.g. `total_reward` that is the total reward for the entire episode).

At the end of the data gathering, you should be able to use pandas (or similar) to calculate the average total reward *per episode* of the random agent. The average score should be 1-2%, meaning that the agent very rarely wins.


## Hints

- `initial_observation = env.reset()` starts a new episode and returns the initial observation.
- `new_observation, reward, done, info = env.step(new_action)` executes one action and returns the following observation. You may look at the documentation for the step method if you are curious about what it does. 
- `done != True` until the game is finished.
- we are trying to maximize the reward *per episode*. Our first game gives 0 reward unless the agent travels to the goal.
- `env.action_space.n` gives the number of possible actions in the environment. `env.action_space.sample()` allows the agent to randomly sample an action.
- `env.observation_space.n` gives the number of possible states in the environment. 

In [8]:
env = gym.make('FrozenLake-v0')

num_episodes = 40000

life_memory = []
for i in range(num_episodes):
    
    # start a new episode and record all the memories
    old_observation = env.reset()
    done = False
    tot_reward = 0
    ep_memory = []
    while not done:
        new_action = env.action_space.sample()
        observation, reward, done, info = env.step(new_action)
        tot_reward += reward
        
        ep_memory.append({
            "observation": old_observation,
            "action": new_action,
            "reward": reward,
            "episode": i,
        })
        old_observation = observation
        
    # incorporate total reward
    num_steps = len(ep_memory)
    for i, ep_mem in enumerate(ep_memory):
        ep_mem["tot_reward"] = tot_reward
        ep_mem["decay_reward"] = i*tot_reward/num_steps
        
    life_memory.extend(ep_memory)
    
memory_df = pandas.DataFrame(life_memory)

In [9]:
memory_df.describe()

,observation,action,reward,episode,tot_reward,decay_reward
count,306568.000000,306568.000000,306568.000000,306568.000000,306568.000000,306568.000000
mean,2.223781,1.499566,0.001853,19974.474446,0.025061,0.011604
std,2.994012,1.117677,0.043004,11493.026500,0.156312,0.085557
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,10084.750000,0.000000,0.000000
50%,1.000000,2.000000,0.000000,19895.000000,0.000000,0.000000
75%,4.000000,2.000000,0.000000,29911.000000,0.000000,0.000000
max,14.000000,3.000000,1.000000,39999.000000,1.000000,0.981818


In [10]:
memory_df.shape

(306568, 6)

In [11]:
memory_df.groupby("episode").reward.sum().mean()

0.0142

# Step 2: Predict

Now that you have a bunch of data, put it into a format that you can model. The goal here is to guide the behavior of our agent. Our agent will be given an observation and need to decide between the possible actions given that observation and the prediction of the model. 

Remember, you're a data scientist! Be creative. 

It might be helpful to work backwards. Ultimately, you will write something like:

```
def convert_to_row(obs, act):
    # expertly written code
    return row_of_obs_act
    
rows = [convert_to_row(current_obs, act) for act in possible_actions]

pred_outcome = model.predict(rows)
```

So, you will need to design a quantity that you can ask your model to predict for every possible action-observation pair. Think a bit about what this quantity should be. Should the model try to predict the immediate reward for each action? If so, how would it know where to go at the beginning of each episode when all moves give zero reward but when some moves bring it closer to the goal than others. 

In [12]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVR

model = ExtraTreesRegressor(n_estimators=50)
# model = SVR()
y = 0.5*memory_df.reward + 0.1*memory_df.decay_reward + memory_df.tot_reward
x = memory_df[["observation", "action"]]
model.fit(x, y)

ExtraTreesRegressor(n_estimators=50)

# Step 3: Act

Now that you have a model that predicts the desired behavior, let's act on it! Modify the code you used to gather data so that you replace the random decision with an intelligent one.

We started out winning ~1.5% of the games with the random agent. How well can you do? You should be able to get your model to do at least 10x better (so 15%). Can you get ~50%?

If you're having trouble, tune your model. Try different representations of the observation and action spaces. Try different models. 

In [15]:
model = RandomForestRegressor()
y = 1*memory_df.reward + memory_df.tot_reward + .1*memory_df.decay_reward
x = memory_df[["observation", "action"]]
x = x.to_numpy()
model.fit(x, y)
print("Model fit completed...")
num_episodes = 500
random_per = 0

life_memory = []
for i in tqdm(range(num_episodes)):
    # start a new episode and record all the memories
    old_observation = env.reset()
    done = False
    tot_reward = 0
    ep_memory = []
    while not done:
        if np.random.rand() < random_per:
            new_action = env.action_space.sample()
        else:
            pred_in = [[old_observation,i] for i in range(4)]
            new_action = np.argmax(model.predict(pred_in))
        observation, reward, done, info = env.step(new_action)
        tot_reward += reward
        
        ep_memory.append({
            "observation": old_observation,
            "action": new_action,
            "reward": reward,
            "episode": i,
        })
        old_observation = observation
        
    # incorporate total reward
    for ep_mem in ep_memory:
        ep_mem["tot_reward"] = tot_reward
        
    life_memory.extend(ep_memory)
    
memory_df2 = pandas.DataFrame(life_memory)

# rf.fit(memory_df[["observation", "action"]], memory_df["comb_reward"])

# score
# much better!
memory_df2.groupby("episode").reward.sum().mean()

Model fit completed...


100%|██████████| 500/500 [02:29<00:00,  3.34it/s]


0.508

In [16]:
y = .1*memory_df.reward + 1*memory_df.decay_reward + 1*memory_df.tot_reward

# Extension: Pole cart

If time permits, try your hand at pole cart (`env = gym.make('CartPole-v0')`).

Notice that the observation space is quite different. It's no longer discrete--instead we have 4 continuous values. You'll have to store these differently from how you did with Frozenlake.

My random actor actually does surprisingly well (avg ~22). But my intelligent agent is able to score ~99. Can you beat me? 

# Pole cart

In [17]:
env = gym.make('CartPole-v0')

In [18]:
# now we can build a toy world!
num_episodes = 1000

life_memory = []
for i in tqdm(range(num_episodes)):
    
    # start a new episode and record all the memories
    old_observation = env.reset()
    done = False
    tot_reward = 0
    ep_memory = []
    while not done:
        new_action = env.action_space.sample()
        observation, reward, done, info = env.step(new_action)
        tot_reward += reward
        
        ep_memory.append({
            "obs0": old_observation[0],
            "obs1": old_observation[1],
            "obs2": old_observation[2],
            "obs3": old_observation[3],
            "action": new_action,
            "reward": reward,
            "episode": i,
        })
        old_observation = observation
        
    # incorporate total reward
    for ep_mem in ep_memory:
        ep_mem["tot_reward"] = tot_reward
        
    life_memory.extend(ep_memory)
    
memory_df = pandas.DataFrame(life_memory)

memory_df.groupby("episode").reward.sum().mean()

100%|██████████| 1000/1000 [00:00<00:00, 2809.90it/s]


22.391

In [19]:
memory_df.describe()

,obs0,obs1,obs2,obs3,action,reward,episode,tot_reward
count,22391.000000,22391.000000,22391.000000,22391.000000,22391.000000,22391.0,22391.000000,22391.000000
mean,0.001356,-0.005356,0.001634,0.012532,0.498415,1.0,488.030459,28.989013
std,0.098823,0.527704,0.091414,0.776796,0.500009,0.0,290.427758,16.467155
min,-0.770615,-2.301722,-0.209299,-2.830750,0.000000,1.0,0.000000,8.000000
25%,-0.040073,-0.360994,-0.051972,-0.489601,0.000000,1.0,230.000000,17.000000
50%,0.000586,-0.002129,0.001848,0.007020,0.000000,1.0,487.000000,25.000000
75%,0.041361,0.350022,0.055883,0.528966,1.000000,1.0,740.000000,36.000000
max,1.360358,2.445054,0.209319,2.950021,1.000000,1.0,999.000000,112.000000


In [22]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor

model = ExtraTreesRegressor(n_estimators=50)

memory_df["comb_reward"] = .5*memory_df.reward + memory_df.tot_reward
model.fit(memory_df[["obs0", "obs1", "obs2", "obs3", "action"]].to_numpy(), memory_df.comb_reward)

ExtraTreesRegressor(n_estimators=50)

In [23]:
num_episodes = 1000
random_per = 0

life_memory = []
for i in tqdm(range(num_episodes)):
    
    # start a new episode and record all the memories
    old_observation = env.reset()
    done = False
    tot_reward = 0
    ep_memory = []
    while not done:
        
        
        if np.random.rand() < random_per:
            new_action = env.action_space.sample()
        else:
            pred_in = [list(old_observation)+[i] for i in range(2)]
            new_action = np.argmax(model.predict(pred_in))
        observation, reward, done, info = env.step(new_action)
        tot_reward += reward
        
        ep_memory.append({
            "obs0": old_observation[0],
            "obs1": old_observation[1],
            "obs2": old_observation[2],
            "obs3": old_observation[3],
            "action": new_action,
            "reward": reward,
            "episode": i,
        })
        old_observation = observation
        
    # incorporate total reward
    for ep_mem in ep_memory:
        ep_mem["tot_reward"] = tot_reward
        
    life_memory.extend(ep_memory)
    
memory_df2 = pandas.DataFrame(life_memory)
memory_df2["comb_reward"] = memory_df2.reward + memory_df2.tot_reward

# score
# much better!
memory_df2.groupby("episode").reward.sum().mean()

100%|██████████| 1000/1000 [09:56<00:00,  1.68it/s]


117.471

---
### Machine Learning Foundation (C) 2020 IBM Corporation